In [20]:
# Imports
from scapy.all import *
from os import path
from os.path import exists
import sys
import ssl
import http.server
import socketserver
import http.client

In [21]:
# Settings
port = 3000
fallback = 'fallback.html'
snapshot_amount = 5
localpath = sys.path[0]

URL = 'paypal.com' #HTTPS URL to use for SSL example
UseURL = False  #Toggle to serving saved snapshot HTMLs if false

In [22]:
# HTTP Request handling and serving
class HttpHandler(http.server.SimpleHTTPRequestHandler):
    def _set_headers(self): # Set up the header
        self.send_response(200)
        self.send_header('Content-Type', 'text/html')
        self.send_header('Content-Length', path.getsize(self.getSnapshot()))
        self.end_headers()
        
    def getSnapshot(self): #Retrieve the fallback page or latest available snapshot
        latest_snapshot = path.join(sys.path[0], fallback)
        for i in range(1, snapshot_amount):
            if exists( path.join(sys.path[0] + '\\Snapshots\\Snapshot_' + str(i) + '.html') ):
                latest_snapshot = path.join(sys.path[0] + '\\Snapshots\\Snapshot_' + str(i) + '.html')
        print('Latest snapshot found at ', latest_snapshot)
        return latest_snapshot
    
    def getHtml(self, htmlPath): # Parse the html into bytes
        with open(htmlPath, mode='r', encoding='utf-8') as openedHTML:
            html = openedHTML.read()
        return bytes(html, 'utf-8')
    
    def do_GET(self): # Serve the html
        self._set_headers()
        if UseURL:
            self.wfile.write(bytes(HttpsGet('paypal.com').read(), 'utf-8'))
        else:
            self.wfile.write(self.getHtml(self.getSnapshot()))
        

In [23]:
# HTTPS Requesting and receiving
def HttpsGet(URL): #Connect using HTTPS to the provided URL
    HttpsConnect = http.client.HTTPSConnection(URL, 443)
    HttpsConnect.putrequest('GET', '/')
    HttpsConnect.endheaders()
    HttpsResponse = HttpsConnect.getresponse()
    return HttpsResponse

In [ ]:
# Attacker server hosting locally
AttackerHandler = HttpHandler
with socketserver.TCPServer(("", port), AttackerHandler) as httpd:
    print("Local server started at localhost:", port)
    httpd.serve_forever()

Local server started at localhost: 3000


127.0.0.1 - - [20/Jun/2023 14:32:29] "GET / HTTP/1.1" 200 -


Latest snapshot found at  D:\Onedrive\OCS Python\Snapshots\Snapshot_1.html
Latest snapshot found at  D:\Onedrive\OCS Python\Snapshots\Snapshot_1.html


127.0.0.1 - - [20/Jun/2023 14:32:31] "GET / HTTP/1.1" 200 -


Latest snapshot found at  D:\Onedrive\OCS Python\Snapshots\Snapshot_1.html
Latest snapshot found at  D:\Onedrive\OCS Python\Snapshots\Snapshot_1.html


In [7]:
# Execute all these (Unfunctional, to be implemented afte fixing HTTPS to HTTP serving)
print( HttpsGet('paypal.com'))
